In [5]:
import random
import matplotlib.pyplot as plt
import networkx as nx

In [6]:
def create_random_graph(num_nodes, prob_edge):
    """Creates a random graph with `num_nodes` nodes and a probability of `prob_edge` for an edge between any two nodes."""
    graph = nx.Graph()
    for i in range(num_nodes):
        graph.add_node(i)
        for j in range(i):
            if random.random() < prob_edge:
                graph.add_edge(i, j)
    return graph

def calculate_modularity(graph, partition, m):
    """Calculates the modularity of a partition of a graph."""
    modularity = 0
    for c in set(partition.values()):
        nodes = [n for n in partition.keys() if partition[n] == c]
        subgraph = graph.subgraph(nodes)
        lc = subgraph.size(weight='weight') / (2*m)
        kc = sum(dict(subgraph.degree(weight='weight')).values()) / (2*m)
        modularity += lc - kc**2
    return modularity

def get_modularity_gain(graph, partition, community, node, degree, degree_community, m):
    """Calculates the modularity gain of moving a node to a new community."""
    delta_q = 0
    for neighbor in graph.neighbors(node):
        if partition[neighbor] == community:
            a_ij = graph[node][neighbor].get('weight', 1)
            delta_q += a_ij - degree*degree_community/(2*m)
    return delta_q / (2*m)

def leiden_algorithm(graph, num_nodes, prob_edge):
    """Applies the Leiden algorithm to a graph."""
    # initialize partition and modularity
    partition = {i:i for i in range(num_nodes)}
    m = graph.size(weight='weight') / 2
    best_partition = partition.copy()
    best_modularity = calculate_modularity(graph, partition, m)
    initial_pass = True
    
    while True:
        # repeat until no further improvement can be made
        improvement = True
        while improvement:
            improvement = False
            
            # get communities
            communities = set(partition.values())
            
            for c in communities:
                # get nodes in community c
                nodes = [n for n in partition.keys() if partition[n] == c]
                subgraph = graph.subgraph(nodes)
                degree_community = subgraph.size(weight='weight')
                
                # calculate modularity gain for moving nodes to other communities
                for node in nodes:
                    degree = graph.degree(node, weight='weight')
                    max_delta_q = 0
                    best_community = c
                    for neighbor in graph.neighbors(node):
                        neighbor_community = partition[neighbor]
                        if neighbor_community == c:
                            continue
                        delta_q = get_modularity_gain(graph, partition, neighbor_community, node, degree, degree_community, m)
                        if delta_q > max_delta_q:
                            max_delta_q = delta_q
                            best_community = neighbor_community
                    
                    # move node to community with highest modularity gain
                    if best_community != c:
                        partition[node] = best_community
                        improvement = True
            
        # create new graph with communities as nodes and edges as weights
        communities = set(partition.values())
        new_graph = nx.Graph()
        for community in communities:
            nodes = [n for n in partition.keys() if partition[n] == community]
            subgraph = graph.subgraph

In [7]:
num_nodes = 50
prob_edge = 0.1

graph = create_random_graph(num_nodes, prob_edge)

final_graph, final_partition, modularity = leiden_algorithm(graph, num_nodes, prob_edge)

# plot final partition
pos = nx.spring_layout(final_graph)
colors = [final_partition[n] for n in final_graph.nodes()]
nx.draw(final_graph, pos, node_color=colors, with_labels=True)
plt.show()

print("Final modularity:", modularity)

KeyboardInterrupt: 